In [1]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd

# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'server' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://server:8181/catalog"
WAREHOUSE = "demo"

SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.6.1"

In [2]:
import json
t = !curl -s -H 'Content-Type: application/x-www-form-urlencoded' --url 'http://keycloak:8080/realms/iceberg/protocol/openid-connect/token' \
  -d 'grant_type=client_credentials&client_id=spark&client_secret=2OR3eRvYfSZzzZ16MlPd95jhLnOaLM52'
t=json.loads(t[0])['access_token']

In [3]:
import requests
headers = {"Authorization":f"Bearer {t}",
                                                                         "Content-Type":"application/json"}


requests.post("http://server:8181/management/v1/bootstrap", headers=headers, json={"accept-terms-of-use": True})
requests.post("http://server:8181/management/v1/warehouse", headers=headers,
                                                            json={
                                                                "warehouse-name": "wh3",
                                                                "storage-profile": {
                "type": "hdfs",
                                                                    "prefix":"/user/hdfs",
                                                                    "url": "hdfs://namenode:8020"
                
                                                            }}).content




b'{"error":{"message":"Failed to write file. Please check the storage credentials.","type":"FileWrite","code":424,"stack":["019625fa-46e3-7af0-afde-b287cd0c737d"]}}'

# Connect with Spark

In [5]:
catalog_name = "lakekeeper"
config = {
    f"spark.sql.catalog.lakekeeper": "org.apache.iceberg.spark.SparkCatalog",
    f"spark.sql.catalog.lakekeeper.type": "rest",
    f"spark.sql.catalog.lakekeeper.uri": CATALOG_URL,
    f"spark.sql.catalog.lakekeeper.warehouse": "wh3",
    f"spark.sql.catalog.{catalog_name}.credential": f"spark:2OR3eRvYfSZzzZ16MlPd95jhLnOaLM52",
    f"spark.sql.catalog.{catalog_name}.scope": "lakekeeper",
    f"spark.sql.catalog.{catalog_name}.oauth2-server-uri": "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token",
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.defaultCatalog": "lakekeeper",
    "spark.jars.packages": f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}",
}


In [6]:
spark_config = SparkConf().setMaster('local').setAppName("Iceberg-REST")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark = SparkSession.builder.config(conf=spark_config).getOrCreate()
spark.sparkContext.setLogLevel("DEBUG")

spark.sql("USE lakekeeper")

DataFrame[]

## Read and Write Tables

In [7]:
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS my_namespace")
spark.sql("SHOW NAMESPACES").toPandas()

,namespace
0,my_namespace


In [8]:
data = pd.DataFrame([[1, 'a-string', 2.2]], columns=['id', 'strings', 'floats'])
sdf = spark.createDataFrame(data)

In [9]:
sdf.writeTo(f"my_namespace.my_tabl2e").createOrReplace()

In [11]:
spark.sql(f"SELECT * FROM my_namespace.my_tabl2e").toPandas()

,id,strings,floats
0,1,a-string,2.2


In [8]:
import requests

In [11]:
requests.get("http://server:8181/catalog/v1/config?warehouse=demo").json()

{'overrides': {'prefix': 'aa93724e-13d3-11f0-940c-079a04f8121a',
  's3.signer.uri': 'http://server:8181/catalog/v1/aa93724e-13d3-11f0-940c-079a04f8121a',
  'uri': 'http://server:8181/catalog'},
 'defaults': {'s3.delete-enabled': 'false'},
 'endpoints': ['POST /v1/aws/s3/sign',
  'POST /v1/{prefix}/v1/aws/s3/sign',
  'GET /v1/config',
  'GET /v1/{prefix}/namespaces',
  'POST /v1/{prefix}/namespaces',
  'GET /v1/{prefix}/namespaces/{namespace}',
  'HEAD /v1/{prefix}/namespaces/{namespace}',
  'POST /v1/{prefix}/namespaces/{namespace}',
  'DELETE /v1/{prefix}/namespaces/{namespace}',
  'POST /v1/{prefix}/namespaces/{namespace}/properties',
  'GET /v1/{prefix}/namespaces/{namespace}/tables',
  'POST /v1/{prefix}/namespaces/{namespace}/tables',
  'GET /v1/{prefix}/namespaces/{namespace}/tables/{table}',
  'POST /v1/{prefix}/namespaces/{namespace}/tables/{table}',
  'DELETE /v1/{prefix}/namespaces/{namespace}/tables/{table}',
  'HEAD /v1/{prefix}/namespaces/{namespace}/tables/{table}',
  'GE

In [41]:
requests.get("http://server:8181/catalog/v1/aa93724e-13d3-11f0-940c-079a04f8121a/namespaces?returnUuids=true&returnProtectionStatus=true").json()

{'namespaces': [], 'namespace-uuids': [], 'protection-status': []}

In [32]:
requests.post("http://server:8181/management/v1/warehouse/aa93724e-13d3-11f0-940c-079a04f8121a/namespace/01961142-e903-76f3-a3c9-79bcd35e9d12/protection", headers={"Content-Type": "application/json"}, json={"protected": True}).json()




{'protected': True,
 'entity_id': '01961142-e903-76f3-a3c9-79bcd35e9d12',
 'updated_at': '2025-04-07T18:41:38.332040Z'}

In [35]:
requests.delete("http://server:8181/catalog/v1/aa93724e-13d3-11f0-940c-079a04f8121a/namespaces")

<Response [405]>

In [40]:
requests.delete("http://server:8181/catalog/v1/aa93724e-13d3-11f0-940c-079a04f8121a/namespaces/my_namespace?recursive=true&force=true").content





b''